In [1]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
nltk.download('omw-1.4')

from itertools import combinations


nltk.download('wordnet_ic')
from nltk.corpus import wordnet_ic
brown_ic = wordnet_ic.ic('ic-brown.dat')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alber\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\alber\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet_ic to
[nltk_data]     C:\Users\alber\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!


In [2]:
# dog.lowest_common_hypernyms(cat)  👉  [Synset('carnivore.n.01')]
# dog.path_similarity(cat)  👉  0.2
# dog.lch_similarity(cat)  👉  2.0281482472922856
# dog.wup_similarity(cat)  👉  0.8571428571428571
# nltk.download('wordnet_ic')
# from nltk.corpus import wordnet_ic
# brown_ic = wordnet_ic.ic('ic-brown.dat')
# dog.lin_similarity(cat,brown_ic)

In [3]:
ls_pairs = [('the','DT'), ('man','NN'), ('swim','VB'), ('with', 'PR'), ('a', 'DT'),
            ('girl','NN'), ('and', 'CC'), ('a', 'DT'), ('boy', 'NN'), ('whilst', 'PR'),
            ('the', 'DT'), ('woman', 'NN'), ('walk', 'VB')]

d_wn_pos = {'NN': wn.NOUN, 'VB': wn.VERB, 'JJ': wn.ADJ, 'RB': wn.ADV}
ls_pairs_wn = [(pair[0], d_wn_pos.get(pair[1])) for pair in ls_pairs if pair[1] in d_wn_pos]
ls_pairs_wn

[('man', 'n'),
 ('swim', 'v'),
 ('girl', 'n'),
 ('boy', 'n'),
 ('woman', 'n'),
 ('walk', 'v')]

# Most Frequent Wordnet Synset

In [4]:
def get_synsets(ls_pairs_wn):
    d_synsets = {}
    for pair in ls_pairs_wn:
        synsets = wn.synsets(pair[0], pair[1])
        if synsets:
            d_synsets[pair[0]] = synsets[0]
        else:
            d_synsets[pair[0]] = None
    return d_synsets

d_synsets = get_synsets(ls_pairs_wn)
d_synsets

{'man': Synset('man.n.01'),
 'swim': Synset('swim.v.01'),
 'girl': Synset('girl.n.01'),
 'boy': Synset('male_child.n.01'),
 'woman': Synset('woman.n.01'),
 'walk': Synset('walk.v.01')}

# Get LCS

In [5]:
d_synsets['walk'].lowest_common_hypernyms(d_synsets['woman'])

[]

In [6]:
word1 = 'man'
word2 = 'girl'

d_synsets[word1].lowest_common_hypernyms(d_synsets[word2])

[Synset('adult.n.01')]

In [7]:
def get_lcs(syn1, syn2):
    return syn1.lowest_common_hypernyms(syn2)

def get_all_lcs(ls_pairs_wn):
    ls_pairs_comb = list(combinations(ls_pairs_wn, 2))
    ls_d_pairs_comb = [get_synsets(comb) for comb in ls_pairs_comb]
    d_lcs = {tuple(d.values()): get_lcs(*list(d.values())) for d in ls_d_pairs_comb}
    return d_lcs

d_lcs_ = get_all_lcs(ls_pairs_wn)
d_lcs_

{(Synset('man.n.01'), Synset('swim.v.01')): [],
 (Synset('man.n.01'), Synset('girl.n.01')): [Synset('adult.n.01')],
 (Synset('man.n.01'), Synset('male_child.n.01')): [Synset('male.n.02')],
 (Synset('man.n.01'), Synset('woman.n.01')): [Synset('adult.n.01')],
 (Synset('man.n.01'), Synset('walk.v.01')): [],
 (Synset('swim.v.01'), Synset('girl.n.01')): [],
 (Synset('swim.v.01'), Synset('male_child.n.01')): [],
 (Synset('swim.v.01'), Synset('woman.n.01')): [],
 (Synset('swim.v.01'), Synset('walk.v.01')): [Synset('travel.v.01')],
 (Synset('girl.n.01'), Synset('male_child.n.01')): [Synset('person.n.01')],
 (Synset('girl.n.01'), Synset('woman.n.01')): [Synset('woman.n.01')],
 (Synset('girl.n.01'), Synset('walk.v.01')): [],
 (Synset('male_child.n.01'), Synset('woman.n.01')): [Synset('person.n.01')],
 (Synset('male_child.n.01'), Synset('walk.v.01')): [],
 (Synset('woman.n.01'), Synset('walk.v.01')): []}


dog.path_similarity(cat)  👉  0.2
dog.lch_similarity(cat)  👉  2.0281482472922856
dog.wup_similarity(cat)  👉  0.8571428571428571
nltk.download('wordnet_ic')
from nltk.corpus import wordnet_ic
brown_ic = wordnet_ic.ic('ic-brown.dat')
dog.lin_similarity(cat,brown_ic) 

In [8]:
import pandas as pd
df = pd.DataFrame(d_lcs_.values(), index=d_lcs_.keys(), columns=['lcs']).reset_index()
df.rename(columns={'level_0': 'word1', 'level_1':'word2'}, inplace=True)
df

,word1,word2,lcs
0,Synset('man.n.01'),Synset('swim.v.01'),None
1,Synset('man.n.01'),Synset('girl.n.01'),Synset('adult.n.01')
2,Synset('man.n.01'),Synset('male_child.n.01'),Synset('male.n.02')
3,Synset('man.n.01'),Synset('woman.n.01'),Synset('adult.n.01')
4,Synset('man.n.01'),Synset('walk.v.01'),None
5,Synset('swim.v.01'),Synset('girl.n.01'),None
6,Synset('swim.v.01'),Synset('male_child.n.01'),None
7,Synset('swim.v.01'),Synset('woman.n.01'),None
8,Synset('swim.v.01'),Synset('walk.v.01'),Synset('travel.v.01')
9,Synset('girl.n.01'),Synset('male_child.n.01'),Synset('person.n.01')


In [9]:
def get_path_similarity(syn1, syn2):
    return syn1.path_similarity(syn2)

def get_lch_similarity(syn1, syn2):
    try:
        return syn1.lch_similarity(syn2)
    except:
        return None
    
def get_wup_similarity(syn1, syn2):
    try:
        return syn1.wup_similarity(syn2)
    except:
        return None
    
def get_lin_similarity(syn1, syn2):
    try:
        return syn1.lin_similarity(syn2, brown_ic)
    except:
        return None

In [10]:
df['path_sim'] = df.apply(lambda x: x['word1'].path_similarity(x['word2']), axis=1)
df['lch_sim'] = df.apply(lambda x: get_lch_similarity(x['word1'], x['word2']), axis=1)
df['wup_sim'] = df.apply(lambda x: get_wup_similarity(x['word1'], x['word2']), axis=1)
df['lin_sim'] = df.apply(lambda x: get_lin_similarity(x['word1'], x['word2']), axis=1)

df

,word1,word2,lcs,path_sim,lch_sim,wup_sim,lin_sim
0,Synset('man.n.01'),Synset('swim.v.01'),None,0.100000,NaN,0.181818,NaN
1,Synset('man.n.01'),Synset('girl.n.01'),Synset('adult.n.01'),0.250000,2.251292,0.631579,0.713511
2,Synset('man.n.01'),Synset('male_child.n.01'),Synset('male.n.02'),0.333333,2.538974,0.666667,0.729472
3,Synset('man.n.01'),Synset('woman.n.01'),Synset('adult.n.01'),0.333333,2.538974,0.666667,0.787084
4,Synset('man.n.01'),Synset('walk.v.01'),None,0.100000,NaN,0.181818,NaN
5,Synset('swim.v.01'),Synset('girl.n.01'),None,0.090909,NaN,0.166667,NaN
6,Synset('swim.v.01'),Synset('male_child.n.01'),None,0.100000,NaN,0.181818,NaN
7,Synset('swim.v.01'),Synset('woman.n.01'),None,0.100000,NaN,0.181818,NaN
8,Synset('swim.v.01'),Synset('walk.v.01'),Synset('travel.v.01'),0.333333,2.159484,0.333333,0.491005
9,Synset('girl.n.01'),Synset('male_child.n.01'),Synset('person.n.01'),0.166667,1.845827,0.631579,0.292728


**Shortest Path Length**: 
$$
Sim(s1,s2) = \dfrac{1}{SPL(s1,s2)}
$$

where SPL(s1,s2) is the Shortest Path Length from s1 to s2 as vertex-countings (similarity of a word and itself is 1) so that 

$$
0 < Sim(s1,s2) \leq 1
$$

**Leacock & Chodorow**: 

$$
Sim(s1,s2) = −\log{\dfrac{SPL(s1,s2)}{2·MaxDepth}}
$$

where depth(s) = SPL(TopSynset,s) and $MaxDepth = \max_{s\in WN}depth(s)$

It is possible to show that there's a maximum value for the similarity of two synsets, which is $\log{2·MaxDepth}$

$$
Sim(s1,s2) = −\log{\dfrac{SPL(s1,s2)}{2·MaxDepth}} = -\log{SPL(s1,s2)} + \log{(2MaxDepth)} \leq \log(2MaxDepth)
$$

so that

$$
0 < \dfrac{Sim(s1,s2)}{\log\left(2MaxDepth\right)} \leq 1
$$


In [12]:
import numpy as np
max_depth = max(max(len(hyp_path) for hyp_path in ss.hypernym_paths()) for ss in wn.all_synsets())
norm_value = np.log(2 * max_depth)

print(f"The max depth is {max_depth} and the normalization value is {norm_value}")

The max depth is 20 and the normalization value is 3.6888794541139363


**Wu & Palmer**:

$$
Sim(s1,s2) = \dfrac{2*depth(LCS(s1,s2))}{depth_{LCS(s1,s2)}(s1)+depth_{LCS(s1,s2)}(s2)}
$$

where LCS(s1,s2) = Lowest Common Subsumer of s1 and s2 and $depth_{s'}(s) = SPL(TopSynset,s)$ through s'

so the highest similarity would be when $depth_{LCS(s1,s2)}(s1)=depth_{LCS(s1,s2)}(s2)$, which is when s1 and s2 are siblings and $depth(LCS(s1,s2)) = maxDepth$. So the maximum value for the similarity of two synsets is 1.


**Lin**: 

$$
Sim(s1,s2) = \dfrac{2*IC(LCS(s1,s2))}{IC(s1)+IC(s2)}
$$

where $IC(s) = −log_2(P(s))$ = information content of s (from frequencies in a corpus)

In the case of Lin, the maximum value for the similarity of two synsets is 1 (when $IC(s1)=IC(s2)$ and $IC(LCS(s1,s2)) = IC(s1) = IC(s2)$)

In [13]:
df_norm = df.copy()
df_norm['lch_sim'] = df_norm['lch_sim'] / norm_value
df_norm = df_norm.loc[:, df.columns]
df_norm

,word1,word2,lcs,path_sim,lch_sim,wup_sim,lin_sim
0,Synset('man.n.01'),Synset('swim.v.01'),None,0.100000,NaN,0.181818,NaN
1,Synset('man.n.01'),Synset('girl.n.01'),Synset('adult.n.01'),0.250000,0.610292,0.631579,0.713511
2,Synset('man.n.01'),Synset('male_child.n.01'),Synset('male.n.02'),0.333333,0.688278,0.666667,0.729472
3,Synset('man.n.01'),Synset('woman.n.01'),Synset('adult.n.01'),0.333333,0.688278,0.666667,0.787084
4,Synset('man.n.01'),Synset('walk.v.01'),None,0.100000,NaN,0.181818,NaN
5,Synset('swim.v.01'),Synset('girl.n.01'),None,0.090909,NaN,0.166667,NaN
6,Synset('swim.v.01'),Synset('male_child.n.01'),None,0.100000,NaN,0.181818,NaN
7,Synset('swim.v.01'),Synset('woman.n.01'),None,0.100000,NaN,0.181818,NaN
8,Synset('swim.v.01'),Synset('walk.v.01'),Synset('travel.v.01'),0.333333,0.585404,0.333333,0.491005
9,Synset('girl.n.01'),Synset('male_child.n.01'),Synset('person.n.01'),0.166667,0.500376,0.631579,0.292728
